# プロジェクトルート固定 & パス定義

In [2]:
from pathlib import Path

# ★リポジトリの絶対パスを明示（必要に応じてご自身の環境に合わせてください）
PROJECT_ROOT = Path(r"C:\Users\user\Desktop\Git\ark-blueprints")

# データ入出力パス（以降のセルはこれを使う）
RAW_DIR   = PROJECT_ROOT / "data" / "raw"
OUT_PATH  = PROJECT_ROOT / "data" / "processed" / "master.csv"
AUDIT_DIR = PROJECT_ROOT / "data" / "processed" / "audit"

# 動作確認（notebooks から実行しても、必ずリポジトリ直下を指す）
print("PROJECT_ROOT:", PROJECT_ROOT)
print("RAW_DIR exists?:", RAW_DIR.exists())
print("OUT_PATH will be:", OUT_PATH)
print("AUDIT_DIR will be:", AUDIT_DIR)


PROJECT_ROOT: C:\Users\user\Desktop\Git\ark-blueprints
RAW_DIR exists?: True
OUT_PATH will be: C:\Users\user\Desktop\Git\ark-blueprints\data\processed\master.csv
AUDIT_DIR will be: C:\Users\user\Desktop\Git\ark-blueprints\data\processed\audit


# ファイル一覧取得

In [4]:
import pandas as pd

files = sorted(RAW_DIR.glob("*.csv"))
len(files), files[:]


(7,
 [WindowsPath('C:/Users/user/Desktop/Git/ark-blueprints/data/raw/20250827_raw.csv'),
  WindowsPath('C:/Users/user/Desktop/Git/ark-blueprints/data/raw/20250828_raw.csv'),
  WindowsPath('C:/Users/user/Desktop/Git/ark-blueprints/data/raw/20250829_raw.csv'),
  WindowsPath('C:/Users/user/Desktop/Git/ark-blueprints/data/raw/20250830_raw.csv'),
  WindowsPath('C:/Users/user/Desktop/Git/ark-blueprints/data/raw/20250831_raw.csv'),
  WindowsPath('C:/Users/user/Desktop/Git/ark-blueprints/data/raw/20250901_raw.csv'),
  WindowsPath('C:/Users/user/Desktop/Git/ark-blueprints/data/raw/20250902_raw.csv')])

# 全ファイルを“壊さず”読み込み（UTF-8-SIG, 全列文字列）

In [6]:
frames = []
for p in files:
    dfi = pd.read_csv(
        p, encoding="utf-8-sig",
        dtype=str, keep_default_na=False,  # まずは現物優先
        engine="python"
    )
    dfi["__source_file"] = p.name  # デバッグ用（最終出力には残ってもOK）
    frames.append(dfi)

raw = pd.concat(frames, ignore_index=True, sort=False)
raw.shape, list(raw.columns)[:15]


((5958, 65),
 ['player_id',
  'AB_class',
  'age',
  'weight',
  'team',
  'origin',
  'run_once',
  'F',
  'L',
  'ST_mean',
  'N_winning_rate',
  'N_2rentai_rate',
  'N_3rentai_rate',
  'LC_winning_rate',
  'LC_2rentai_rate'])

# 作業用 df 作成 & 日付型

In [8]:
df = raw.copy()

# date は YYYYMMDD が基本の想定（合わない値は NaT）
if "date" in df.columns:
    try:
        df["date"] = pd.to_datetime(df["date"], format="%Y%m%d", errors="coerce")
    except Exception:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5958 entries, 0 to 5957
Data columns (total 65 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   player_id           5958 non-null   object        
 1   AB_class            5958 non-null   object        
 2   age                 5958 non-null   object        
 3   weight              5958 non-null   object        
 4   team                5958 non-null   object        
 5   origin              5958 non-null   object        
 6   run_once            5958 non-null   object        
 7   F                   5958 non-null   object        
 8   L                   5958 non-null   object        
 9   ST_mean             5958 non-null   object        
 10  N_winning_rate      5958 non-null   object        
 11  N_2rentai_rate      5958 non-null   object        
 12  N_3rentai_rate      5958 non-null   object        
 13  LC_winning_rate     5958 non-null   object      

# 全角→半角（数字・小数点・マイナス）→ 型変換（最小）

In [11]:
import pandas as pd

# 全角→半角（数字・小数点・マイナスのみ）
ZEN2HAN = str.maketrans("０１２３４５６７８９．－", "0123456789.-")

def normalize_zenkaku_digits(s: pd.Series) -> pd.Series:
    if s.dtype != object:
        return s
    return s.str.translate(ZEN2HAN)

# 4a) rank を最優先で正規化→数値化（今回の肝）
converted_report = {}
if "rank" in df.columns:
    df["rank"] = normalize_zenkaku_digits(df["rank"].astype(str))
    out = pd.to_numeric(df["rank"], errors="coerce").astype("Int64")
    converted_report["rank"] = {"target": "int", "n": int(len(out)), "n_nan": int(out.isna().sum())}
    df["rank"] = out

# 4b) ほかにも全角数字が紛れやすい列だけ軽く正規化
maybe_zen_cols = [c for c in ["wakuban","R","entry","entry_tenji","ST_rank","day","section","code"] if c in df.columns]
for c in maybe_zen_cols:
    if df[c].dtype == object:
        df[c] = normalize_zenkaku_digits(df[c].astype(str))

# 4c) “素直に”型変換（ST/ ST_tenji は触らない）
to_int = [
    "player_id","age","run_once","F","L",
    "motor_number","boat_number","pred_mark","code","R",
    "entry_tenji","wakuban","entry","ST_rank",
    "day","section","is_wakunari",
]
to_float = [
    "weight","ST_mean",
    "temperature","wind_speed","water_temperature","wave_height",
    "time_tenji","Tilt",
]

for c in to_int:
    if c in df.columns:
        out = pd.to_numeric(df[c], errors="coerce").astype("Int64")
        converted_report[c] = {"target": "int", "n": int(len(out)), "n_nan": int(out.isna().sum())}
        df[c] = out

for c in to_float:
    if c in df.columns:
        out = pd.to_numeric(df[c], errors="coerce")
        converted_report[c] = {"target": "float", "n": int(len(out)), "n_nan": int(out.isna().sum())}
        df[c] = out

converted_report


{'rank': {'target': 'int', 'n': 5958, 'n_nan': 79},
 'player_id': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'age': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'run_once': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'F': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'L': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'motor_number': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'boat_number': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'pred_mark': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'code': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'R': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'entry_tenji': {'target': 'int', 'n': 5958, 'n_nan': 8},
 'wakuban': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'entry': {'target': 'int', 'n': 5958, 'n_nan': 8},
 'ST_rank': {'target': 'int', 'n': 5958, 'n_nan': 35},
 'day': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'section': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'is_wakunari': {'target': 'int', 'n': 5958, 'n_nan': 0},
 'weight': {'target': 'float', 'n': 59

# rank 非数値（=NaN）を含むレースを race_id 単位で除外

In [13]:
if "race_id" not in df.columns:
    raise ValueError("race_id 列が必要です。")

bad_races_rank = sorted(df.loc[df["rank"].isna(), "race_id"].dropna().unique().tolist())

before_rows, before_races = len(df), df["race_id"].nunique()
df = df[~df["race_id"].isin(bad_races_rank)].copy()
after_rows, after_races = len(df), df["race_id"].nunique()

print(f"[DROP] rank-nonnumeric races: {len(bad_races_rank)}")
print(f"[INFO] rows  {before_rows} -> {after_rows}  (removed {before_rows - after_rows})")
print(f"[INFO] races {before_races} -> {after_races}")


[DROP] rank-nonnumeric races: 64
[INFO] rows  5958 -> 5574  (removed 384)
[INFO] races 993 -> 929


# 気象関連に欠損が1つでもあるレースを除外

In [15]:
wx_cols = [c for c in ["temperature","wind_speed","water_temperature","wave_height","weather","wind_direction"] if c in df.columns]

if wx_cols:
    mask_any_na = df[wx_cols].isna().any(axis=1)
    bad_races_wx = sorted(df.loc[mask_any_na, "race_id"].dropna().unique().tolist())
else:
    bad_races_wx = []

before_rows2, before_races2 = len(df), df["race_id"].nunique()
df = df[~df["race_id"].isin(bad_races_wx)].copy()
after_rows2, after_races2 = len(df), df["race_id"].nunique()

print(f"[DROP] weather-missing races: {len(bad_races_wx)}")
print(f"[INFO] rows  {before_rows2} -> {after_rows2}  (removed {before_rows2 - after_rows2})")
print(f"[INFO] races {before_races2} -> {after_races2}")


[DROP] weather-missing races: 1
[INFO] rows  5574 -> 5568  (removed 6)
[INFO] races 929 -> 928


# rate 系を float 化（0–100のまま）

In [17]:
# rate全列を float 化＋NaNは0.00に置換
import pandas as pd

rate_cols = [
    "N_winning_rate","N_2rentai_rate","N_3rentai_rate",
    "LC_winning_rate","LC_2rentai_rate","LC_3rentai_rate",
    "motor_2rentai_rate","motor_3rentai_rate",
    "boat_2rentai_rate","boat_3rentai_rate",
]

present = [c for c in rate_cols if c in df.columns]
if not present:
    print("[INFO] rate columns not found; skip.")
else:
    # 1) まず float 化（不正値は NaN）
    df[present] = df[present].apply(pd.to_numeric, errors="coerce")

    # 2) NaN を 0.00 に置換（公式の意図に合わせる）
    na_counts_before = df[present].isna().sum()
    total_cells = int(na_counts_before.sum())
    affected_races = sorted(df.loc[df[present].isna().any(axis=1), "race_id"].dropna().unique().tolist())

    if total_cells > 0:
        df[present] = df[present].fillna(0.0)
        print(f"[FIX] rate NaN -> 0.00 : {total_cells} cells  | races={len(affected_races)} -> {affected_races[:10]}{' ...' if len(affected_races)>10 else ''}")
    else:
        print("[INFO] no NaN in rate columns")

    # 3) 簡易チェック
    print("rate dtypes:\n", df[present].dtypes)
    print("remaining NaN counts:\n", df[present].isna().sum().sort_values(ascending=False).head(10))


[FIX] rate NaN -> 0.00 : 294 cells  | races=88 -> ['202508270901', '202508270910', '202508271001', '202508271003', '202508271009', '202508271104', '202508271109', '202508271601', '202508271602', '202508271608'] ...
rate dtypes:
 N_winning_rate        float64
N_2rentai_rate        float64
N_3rentai_rate        float64
LC_winning_rate       float64
LC_2rentai_rate       float64
LC_3rentai_rate       float64
motor_2rentai_rate    float64
motor_3rentai_rate    float64
boat_2rentai_rate     float64
boat_3rentai_rate     float64
dtype: object
remaining NaN counts:
 N_winning_rate        0
N_2rentai_rate        0
N_3rentai_rate        0
LC_winning_rate       0
LC_2rentai_rate       0
LC_3rentai_rate       0
motor_2rentai_rate    0
motor_3rentai_rate    0
boat_2rentai_rate     0
boat_3rentai_rate     0
dtype: int64


# ST / ST_tenji を“符号付き秒”に変換

In [19]:
# ST / ST_tenji を“符号付き秒”に変換、展示はレース内順位も作成（差し替え版）
import numpy as np, re, pandas as pd

def parse_st(s) -> float:
    """
    'F.01' -> -0.01, '0.07' -> +0.07, 'L.03' -> +0.03
    '3  L' -> NaN（= L 単独として扱い、量不明） / '3F.01' -> 'F.01' として解釈
    不正・空欄は NaN
    """
    if s is None:
        return np.nan
    t = str(s).strip()
    if t == "" or t in {"-", "—", "ー", "―"}:
        return np.nan

    # 全角を半角へ（F/L のみ）
    t = t.replace("Ｆ", "F").replace("Ｌ", "L")

    # --- 小さな保険1: 先頭に枠番が紛れた「<digits> [F|L]（.xx 任意）」を末尾の F/L 以降に切り出す
    # 例) "3  L" -> "L" , "3F.01" -> "F.01"
    m = re.match(r"^\d+\s*([FL](?:\.\d+)?)$", t, flags=re.I)
    if m:
        t = m.group(1)  # "L" or "F.01" に正規化

    # 先頭が F/L なら符号を決めて本体を取り出し
    sign = 1.0
    if t[:1].lower() == "f":
        sign = -1.0
        t = t[1:].strip()
    elif t[:1].lower() == "l":
        sign = 1.0
        t = t[1:].strip()

    # --- 小さな保険2: "01" のように小数点がない2桁なら "0.01" に補正
    if re.fullmatch(r"\d{2}", t):
        t = "0." + t

    # 先頭が "." の場合は "0.01" へ
    if t.startswith("."):
        t = "0" + t

    # ここまでで数値部が空（例: "L" 単独）なら NaN
    if t == "":
        return np.nan

    # 数値形式チェック
    if not re.fullmatch(r"\d+(\.\d+)?", t):
        return np.nan

    try:
        val = float(t)
    except ValueError:
        return np.nan
    return sign * val

# 数値化（展示・本番）
if "ST_tenji" in df.columns:
    df["ST_tenji_num"] = df["ST_tenji"].apply(parse_st)

if "ST" in df.columns:
    df["ST_num"] = df["ST"].apply(parse_st)   # ※学習では使わない（将来用）

# レース内順位（展示のみ：小さい=早い → 1位）
if {"race_id","ST_tenji_num"}.issubset(df.columns):
    df["ST_tenji_rank"] = (
        df.groupby("race_id")["ST_tenji_num"]
          .rank(method="min", ascending=True)
          .astype("Int64")
    )

# 動作確認（任意）
print("ST_tenji→num:", list(zip(df.get("ST_tenji", [])[:6], df.get("ST_tenji_num", [])[:6])))
print("ST→num      :", list(zip(df.get("ST", [])[:6],       df.get("ST_num", [])[:6])))


ST_tenji→num: [(' .04', 0.04), ('F.01', -0.01), (' .07', 0.07), (' .03', 0.03), (' .02', 0.02), ('F.02', -0.02)]
ST→num      : [('0.18', 0.18), ('0.18', 0.18), ('0.17', 0.17), ('0.22', 0.22), ('0.23', 0.23), ('0.19', 0.19)]


# ST/ST_tenji を正式カラムとして float 化

In [21]:
# 1) 変換値を上書き反映
if "ST_tenji_num" in df.columns:
    df["ST_tenji"] = df["ST_tenji_num"].astype(float)

if "ST_num" in df.columns:
    df["ST"] = df["ST_num"].astype(float)

# 2) ヘルパー列だけ片付け（変換値は ST/ST_tenji に残る）
for c in ["ST_tenji_num", "ST_num"]:
    if c in df.columns:
        df.drop(columns=c, inplace=True)

# 3) 確認
print(df[["ST_tenji","ST"]].dtypes)         # → float64, float64
print(df[["ST_tenji","ST"]].isna().sum())   # 欠損数チェック
print("ST_tenji_rank dtype:", df["ST_tenji_rank"].dtype if "ST_tenji_rank" in df.columns else "N/A")


ST_tenji    float64
ST          float64
dtype: object
ST_tenji    1
ST          0
dtype: int64
ST_tenji_rank dtype: Int64


# 展示STに非数値があればレースごと除外

In [23]:
# 展示STが非数値（Lのみ等）を含むレースを丸ごと除外
if "ST_tenji" in df.columns:
    mask_bad = df["ST_tenji"].isna()
    bad_races_sttenji = sorted(df.loc[mask_bad, "race_id"].dropna().unique().tolist())
else:
    bad_races_sttenji = []

before_rows, before_races = len(df), df["race_id"].nunique()
df = df[~df["race_id"].isin(bad_races_sttenji)].copy()
after_rows, after_races = len(df), df["race_id"].nunique()

print(f"[DROP] races with non-numeric ST_tenji: {len(bad_races_sttenji)}")
print(f"[INFO] rows  {before_rows} -> {after_rows}  (removed {before_rows - after_rows})")
print(f"[INFO] races {before_races} -> {after_races})")

# （任意）確認用に何件か表示
if bad_races_sttenji:
    preview = df.loc[df["race_id"].isin(bad_races_sttenji), ["race_id"]].drop_duplicates().head()
    print("[SAMPLE DROPPED race_id]:")
    print(preview.to_string(index=False))


[DROP] races with non-numeric ST_tenji: 1
[INFO] rows  5568 -> 5562  (removed 6)
[INFO] races 928 -> 927)
[SAMPLE DROPPED race_id]:
Empty DataFrame
Columns: [race_id]
Index: []


# 目的変数 is_top2 の作成（正常レースのみ）

In [26]:
df["is_top2"] = (df["rank"] <= 2).astype("Int64")
df["is_top2"].value_counts(dropna=False)


is_top2
0    3708
1    1854
Name: count, dtype: Int64

# 除外レース一覧の保存

In [28]:
# 除外レース一覧の保存（差し替え版）— ST展示の非数値ルールを追加
import pandas as pd
from datetime import datetime

REPORTS_DIR = PROJECT_ROOT / "data" / "processed" / "reports"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

RUN_ID = datetime.now().strftime("%Y%m%d-%H%M%S")

# 安全ガード（変数が未定義でも動くように）
bad_races_rank     = bad_races_rank     if 'bad_races_rank'     in locals() else []
bad_races_wx       = bad_races_wx       if 'bad_races_wx'       in locals() else []
bad_races_sttenji  = bad_races_sttenji  if 'bad_races_sttenji'  in locals() else []

# ① 今回の実行での除外レコード
records = []
records += [("rank_nonnumeric",      rid, "着順が数字でない（失格・転覆等）") for rid in bad_races_rank]
records += [("weather_missing_any",  rid, "気象データに欠損あり")           for rid in bad_races_wx]
records += [("sttenji_nonnumeric",   rid, "展示STに数値化できない表記あり（L等）") for rid in bad_races_sttenji]

df_new = pd.DataFrame(records, columns=["rule_key", "race_id", "reason"])
df_new["race_id"] = df_new["race_id"].astype(str)
df_new["run_id"] = RUN_ID

# A) スナップショット
snap_path = REPORTS_DIR / f"excluded_races_{RUN_ID}.csv"
df_new.to_csv(snap_path, index=False, encoding="utf-8-sig")

# B) 集約（過去分と結合→理由をユニーク結合）
agg_path = REPORTS_DIR / "excluded_races.csv"
if agg_path.exists():
    df_hist = pd.read_csv(agg_path, dtype={"race_id": str}, encoding="utf-8-sig")[["race_id","reason"]]
else:
    df_hist = pd.DataFrame(columns=["race_id","reason"])

df_all = pd.concat([df_hist, df_new[["race_id","reason"]]], ignore_index=True)
df_agg = (
    df_all.groupby("race_id")["reason"]
          .apply(lambda s: " / ".join(sorted(set(s.dropna()))))
          .reset_index()
          .sort_values("race_id")
)
df_agg.to_csv(agg_path, index=False, encoding="utf-8-sig")

print("[OK] 除外レース（スナップショット）:", snap_path)
print("[OK] 除外レース（集約）:", agg_path)
print(df_agg.head(10).to_string(index=False))


[OK] 除外レース（スナップショット）: C:\Users\user\Desktop\Git\ark-blueprints\data\processed\reports\excluded_races_20250903-103048.csv
[OK] 除外レース（集約）: C:\Users\user\Desktop\Git\ark-blueprints\data\processed\reports\excluded_races.csv
     race_id           reason
202508270106 着順が数字でない（失格・転覆等）
202508270507 着順が数字でない（失格・転覆等）
202508270510 着順が数字でない（失格・転覆等）
202508270512 着順が数字でない（失格・転覆等）
202508271101       気象データに欠損あり
202508271206 着順が数字でない（失格・転覆等）
202508271710 着順が数字でない（失格・転覆等）
202508272005 着順が数字でない（失格・転覆等）
202508272009 着順が数字でない（失格・転覆等）
202508280106 着順が数字でない（失格・転覆等）


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5562 entries, 0 to 5957
Data columns (total 67 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   player_id           5562 non-null   Int64         
 1   AB_class            5562 non-null   object        
 2   age                 5562 non-null   Int64         
 3   weight              5562 non-null   float64       
 4   team                5562 non-null   object        
 5   origin              5562 non-null   object        
 6   run_once            5562 non-null   Int64         
 7   F                   5562 non-null   Int64         
 8   L                   5562 non-null   Int64         
 9   ST_mean             5562 non-null   float64       
 10  N_winning_rate      5562 non-null   float64       
 11  N_2rentai_rate      5562 non-null   float64       
 12  N_3rentai_rate      5562 non-null   float64       
 13  LC_winning_rate     5562 non-null   float64       
 1

# 最終ミニチェック

In [31]:
assert len(df) % 6 == 0, "行数が6の倍数ではありません"
assert df["is_top2"].isna().sum() == 0, "is_top2にNaNがあります"


# master.csv を UTF-8-SIG で書き出し（Excel互換）

In [33]:
# 保存（UTF-8-SIGでExcelも安心）
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
print("[OK] wrote:", OUT_PATH, df.shape)


[OK] wrote: C:\Users\user\Desktop\Git\ark-blueprints\data\processed\master.csv (5562, 67)
